#### 29.10.25, &copy; [Evhenii Kostin](https://github.com/DE123MasterProgram2025autumn/DE_Kostin), 2025

# Лабораторна робота №1. Основи маніпуляції даними з dplyr

__Мета:__ _освоїти фундаментальні операції маніпуляції даними в середовищі R за допомогою сучасного пакета dplyr, використовуючи функціональний підхід та конвеєрне програмування (%>% або |>). Навчитися ефективно фільтрувати, перетворювати, агрегувати та реструктурувати дані для подальшого аналізу чи завантаження в ETL-процеси._

### Варіант 2
Датасет _mtcars_

#### 1. Вибірка та фільтрація
Відфільтруйте автомобілі, у яких кількість циліндрів (cyl) дорівнює 6 або 8. Виберіть лише стовпці: mpg, hp, wt, cyl, gear. Відсортуйте за спаданням потужності (hp).

#### 2. Створення нового стовпця
Додайте стовпець power_to_weight (співвідношення потужності до ваги), розрахований за формулою: power_to_weight = hp / wt. Оберіть автомобілі, у яких power_to_weight > 50, і відсортуйте їх за спаданням power_to_weight.

#### 3. Групування та агрегація
Для кожної кількості передач (gear), яка представлена більш ніж у двох автомобілів, розрахуйте:

середню витрату палива (mean_mpg), середню потужність (mean_hp), кількість автомобілів (n). Використайте summarise() та across() для агрегації числових стовпців (mpg, hp).

#### 4. Перейменування та реструктуризація
Візьміть результат попереднього пункту.

Перейменуйте стовпець gear на Кількість_передач. Перемістіть стовпець n (кількість) на останнє місце. Перетворіть назви всіх інших стовпців на нижній регістр з підкресленням (наприклад, mean_mpg залишається, але якщо було б MeanMPG — перетворювалося б на mean_mpg).

In [7]:
import pandas as pd
import statsmodels.api as sm # Використовуємо це для надійного завантаження mtcars

# --- Завантаження датасету ---
# Це найнадійніший спосіб завантажити 'mtcars' у Python
mtcars_data = sm.datasets.get_rdataset("mtcars", "datasets", cache=True)
df = mtcars_data.data

print("--- Початковий датафрейм (перші 5 рядків) ---")
print(df.head())


# --- 1. Вибірка та фільтрація ---
print("\n--- Завдання 1: 6/8 циліндрів, сортовано за hp ---")

# Фільтруємо (cyl == 6 або 8)
task1_result = df[df['cyl'].isin([6, 8])]

# Вибираємо стовпці
task1_result = task1_result[['mpg', 'hp', 'wt', 'cyl', 'gear']]

# Сортуємо за спаданням 'hp'
task1_result = task1_result.sort_values(by='hp', ascending=False)

print(task1_result.head())


# --- 2. Створення нового стовпця ---
print("\n--- Завдання 2: power_to_weight > 50 ---")

# Копіюємо, щоб не змінювати оригінал
df_task2 = df.copy()

# Додаємо стовпець power_to_weight
df_task2['power_to_weight'] = df_task2['hp'] / df_task2['wt']

# Фільтруємо > 50 та сортуємо
task2_result = df_task2[df_task2['power_to_weight'] > 50].sort_values(
    by='power_to_weight', ascending=False
)

print(task2_result.head())


# --- 3. Групування та агрегація ---
print("\n--- Завдання 3: Групування за 'gear' > 2 ---")

# 1. Спочатку фільтруємо групи, які мають > 2 автомобілів
# Це еквівалент R: group_by(gear) %>% filter(n() > 2)
grouped_by_gear = df.groupby('gear')
filtered_groups_df = grouped_by_gear.filter(lambda x: len(x) > 2)

# 2. Тепер групуємо відфільтровані дані та агрегуємо
# Це еквівалент R: summarise(across(c(mpg, hp), mean), n = n())
task3_result = filtered_groups_df.groupby('gear').agg(
    mean_mpg=('mpg', 'mean'),  # Розраховуємо середнє для mpg
    mean_hp=('hp', 'mean'),    # Розраховуємо середнє для hp
    n=('gear', 'size')         # Рахуємо кількість (n)
).reset_index() # Перетворюємо 'gear' з індексу на стовпець

print(task3_result)


# --- 4. Перейменування та реструктуризація ---
print("\n--- Завдання 4: Фінальна таблиця ---")

df_task4 = task3_result.copy()

# Перейменовуємо 'gear'
df_task4 = df_task4.rename(columns={'gear': 'Кількість_передач'})

# Переміщуємо 'n' на останнє місце
# Беремо всі стовпці, крім 'n'
cols = [col for col in df_task4.columns if col != 'n']
# Додаємо 'n' в кінець списку та оновлюємо DataFrame
df_task4 = df_task4[cols + ['n']]

# Перетворюємо назви інших стовпців на нижній регістр
# (Вони вже є, але це формальне виконання умови)
cols_to_rename = [col for col in df_task4.columns if col not in ['Кількість_передач', 'n']]
rename_map = {col: col.lower() for col in cols_to_rename}
task4_result = df_task4.rename(columns=rename_map)

print(task4_result)

--- Початковий датафрейм (перші 5 рядків) ---
                    mpg  cyl   disp   hp  drat     wt   qsec  vs  am  gear  \
rownames                                                                     
Mazda RX4          21.0    6  160.0  110  3.90  2.620  16.46   0   1     4   
Mazda RX4 Wag      21.0    6  160.0  110  3.90  2.875  17.02   0   1     4   
Datsun 710         22.8    4  108.0   93  3.85  2.320  18.61   1   1     4   
Hornet 4 Drive     21.4    6  258.0  110  3.08  3.215  19.44   1   0     3   
Hornet Sportabout  18.7    8  360.0  175  3.15  3.440  17.02   0   0     3   

                   carb  
rownames                 
Mazda RX4             4  
Mazda RX4 Wag         4  
Datsun 710            1  
Hornet 4 Drive        1  
Hornet Sportabout     2  

--- Завдання 1: 6/8 циліндрів, сортовано за hp ---
                    mpg   hp     wt  cyl  gear
rownames                                      
Maserati Bora      15.0  335  3.570    8     5
Ford Pantera L     15.8  264  3.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6a083947-94ba-475d-8730-27cff0574f54' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>